In [1]:
import pandas as pd
import numpy as np
from imdb import IMDb

ia = IMDb()
movies_csv_path = "Resources/AllMoviesDetailsCleaned.csv"
streaming_csv_path = "Resources/MoviesOnStreamingPlatforms_updated.csv"

In [2]:
movies_df = pd.read_csv(movies_csv_path, encoding="utf-8", delimiter=";", parse_dates=["release_date"])
streaming_df = pd.read_csv(streaming_csv_path, encoding="utf-8", parse_dates=["Year"])

F:\Programs\Anaconda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies_df["release_date"] = movies_df["release_date"].dt.year
streaming_df["Year"] = streaming_df["Year"].dt.year
movies_df = movies_df.loc[(movies_df["release_date"].isnull() != True),:]
movies_df["release_date"] = movies_df["release_date"].astype(int)

In [ ]:
#streaming_df
movie = ia.get_movie('0266543')
movie['directors'][0]['name']

In [ ]:
movie['genres']

In [4]:
super_merge = pd.merge(movies_df, streaming_df, left_on=["title", "release_date"], right_on=["Title", "Year"]).reset_index(drop=True)
super_merge['genres'] = super_merge['genres'].str.replace(pat="|", repl=",")
desiredColumns = ['title', 'genres', 'Genres', 'release_date', 'budget', 'revenue', 'Runtime', 'Country', 'Language', 'spoken_languages_number', 'Age', 'popularity', 'vote_average', 'vote_count', 'imdb_id', 'IMDb', 'Rotten Tomatoes', 'Directors']
super_merge = super_merge.loc[:, desiredColumns]

#cleaned up imdb_id with a left strip. now this id is usable with the imdb module
super_merge['imdb_id'] = super_merge['imdb_id'].str.lstrip('tt')

#filling NaNs for datagrabbing
super_merge.loc[super_merge['imdb_id'].isnull(),'imdb_id'] = "None"
super_merge.loc[super_merge['genres'].isnull(),'genres'] = "None"
super_merge.loc[super_merge['Genres'].isnull(),'Genres'] = "None"
super_merge.loc[super_merge['Directors'].isnull(),'Directors'] = "None"

#Grabbing only the first director from possible arrays.
directors_list = super_merge['Directors'].str.split(pat=",")
first_director_list = []
for i in directors_list:
    first_director_list.append(i[0])
    
#reassigning the directors column with the scrubbed director list    
super_merge['Directors'] = first_director_list


super_merge["Director_ID"] = ""
super_merge

,title,genres,Genres,release_date,budget,revenue,Runtime,Country,Language,spoken_languages_number,Age,popularity,vote_average,vote_count,imdb_id,IMDb,Rotten Tomatoes,Directors,Director_ID
0,Finding Nemo,"Animation,Family","Animation,Adventure,Comedy,Family",2003,94000000,940335536,100.0,"United States,Australia",English,1,all,9.915573,7.6,5531,0266543,8.1,99%,Andrew Stanton,
1,The Endless Summer,Documentary,"Documentary,Sport",1966,0,0,95.0,United States,English,1,7+,0.144179,7.8,20,0060371,7.7,100%,Bruce Brown,
2,Pirates of the Caribbean: The Curse of the Bla...,"Adventure,Fantasy,Action","Action,Adventure,Fantasy",2003,140000000,655011224,143.0,United States,English,1,13+,28.769026,7.4,6368,0325980,8.0,79%,Gore Verbinski,
3,Kill Bill: Vol. 1,"Action,Crime","Action,Crime,Thriller",2003,30000000,180949000,111.0,"United States,Japan","English,Japanese,French",3,18+,7.891837,7.7,4486,0266697,8.1,85%,Quentin Tarantino,
4,Jarhead,"Drama,War","Action,Biography,Drama,War",2005,72000000,96889998,125.0,"United States,United Kingdom,Germany","English,Spanish,Arabic,Latin",4,18+,2.41718,6.5,722,0418763,7.0,61%,Sam Mendes,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12630,Shot,None,"Adventure,Drama,Horror,Mystery,Thriller",2017,0,0,86.0,Uruguay,Spanish,0,NaN,0.0703,0.0,0,4323568,5.3,56%,Gustavo Hernández,
12631,Tie the Knot,None,"Comedy,Drama,Family,Musical,Romance",2016,0,0,90.0,"United States,India",English,0,NaN,0.5537,0.0,0,3038542,3.3,NaN,Shuja Paul,
12632,Under an Arctic Sky,"Adventure,Documentary","Documentary,Short,Adventure,Sport",2017,0,0,39.0,United States,"Icelandic,English",0,NaN,0.0218,10.0,1,6680858,7.0,NaN,Chris Burkard,
12633,Plug Love,"Crime,Romance,Drama",Drama,2017,0,0,116.0,United States,English,0,NaN,0.1599,7.0,1,None,5.9,NaN,Derek Scott,


In [9]:
imdb_counter = 0

for index,row in super_merge.iterrows():
    if row['Genres'] == "None":
        super_merge.loc[index, 'Genres'] = row['genres']
    if (row['Genres'] == "None") & (row['genres'] == "None") & (row['imdb_id'] != "None"):
        imdb_counter += 1
        try:
            movie = ia.get_movie(row['imdb_id'])
            imdb_genre = movie['genres'][0]
            super_merge.loc[index, 'Genres'] = imdb_genre
        except (KeyError, ValueError):
            print(f'Genre data was not avilable for {index} {row["title"]} Value error with {imdb_genre}')
            
    if (row['Directors'] == 'None') & (row['imdb_id'] != "None"):
        try:
            print(f"Grabbing Director for {row['title']}..")
            movie = ia.get_movie(row['imdb_id'])
            imdb_counter += 1
            director_name = movie['directors'][0]['name']
            super_merge.loc[index, 'Directors'] = director_name
        except KeyError:
            print(f'Director data was not avilable for {index} {row["title"]}')
            
print(f"The IMDb module was used {imdb_counter} times")        

Grabbing Director for St. Paul..
Director data was not avilable for 194 St. Paul
Grabbing Director for Kingdom of the Blue Whale..
Director data was not avilable for 1499 Kingdom of the Blue Whale
Grabbing Director for Combat America..
Director data was not avilable for 1564 Combat America
Grabbing Director for Bollywood Hero..
Director data was not avilable for 1581 Bollywood Hero
Grabbing Director for Tupac Shakur: Thug Immortal..
Director data was not avilable for 2139 Tupac Shakur: Thug Immortal
Grabbing Director for A Christmas Carol..
Grabbing Director for Blackfly..
Grabbing Director for The Locksmith..
Grabbing Director for Untold Scandal..
Grabbing Director for Out of Order..
Director data was not avilable for 3385 Out of Order
Grabbing Director for Shaka Zulu: The Last Great Warrior..
Grabbing Director for Neighbours..
Grabbing Director for Michael Jordan, Above and Beyond..
Director data was not avilable for 3459 Michael Jordan, Above and Beyond
Grabbing Director for Citadel

Director data was not avilable for 8665 A Fall from Freedom
Grabbing Director for Jesus: He Lived Among Us..
Director data was not avilable for 8672 Jesus: He Lived Among Us
Grabbing Director for Glenn Wool: You Don't Go to Hell for Eating Elephants..
Director data was not avilable for 8690 Glenn Wool: You Don't Go to Hell for Eating Elephants
Grabbing Director for The Stilwell Road..
Director data was not avilable for 8745 The Stilwell Road
Grabbing Director for Baja: The Other California..
Director data was not avilable for 8746 Baja: The Other California
Grabbing Director for Murge: The Cold War Front..
Genre data was not avilable for 8809 Paranormal Rosslyn Chapel Value error with Documentary
Grabbing Director for Where Were You When I Needed You..
Genre data was not avilable for 8862 Behind the Race to the Moon Value error with Documentary
Grabbing Director for Behind the Race to the Moon..
Director data was not avilable for 8862 Behind the Race to the Moon
Grabbing Director for M

Grabbing Director for Marco Luque - We are together..
Grabbing Director for Rifftrax Live: Samurai Cop..
Director data was not avilable for 12582 Rifftrax Live: Samurai Cop
Grabbing Director for The Neighborhood..
Grabbing Director for Temple..
The IMDb module was used 301 times


In [12]:
super_merge.loc[(super_merge["Genres"] == "None"), 'Genres'].count()

20

In [ ]:
#super_merge.loc[(super_merge["imdb_id"].isnull()) & (super_merge["Directors"].isnull()), :]

In [ ]:
# code to loop through merged df to give streaming provider id. went a different route so storing code just in case we want to go this route
# for index,row in super_merge.iterrows():
#     if (row['Netflix'] == 1) & (row['Hulu'] == 0) & (row['Prime Video'] == 0) & (row['Disney+'] == 0):
#         super_merge.loc[index, 'Streaming_Provider_ID'] = '1'
#     elif row['Hulu'] == 1:
#         super_merge.loc[index, 'Streaming_Provider_ID'] = '2'
#     elif row['Prime Video'] == 1:
#         super_merge.loc[index, 'Steaming_Provider_ID'] = 3
#     elif row['Disney+'] ==1 :
        
